In [27]:
import numpy as np
import pandas as pd
from pupil_functions import *
import platform
from shutil import copyfile
from find_blob import *

# tracking delay
## TCB_GOOD

In [28]:
col = ["block", "trial", "task", "sent", "received"]
timesTCB = pd.DataFrame(columns = col)

In [29]:
dfTCB = load_manual_annotations("/Users/selena_zhang/Desktop/000")

In [30]:
#change none's to nearest numbers
for idx, num in dfTCB.iloc[:, 4].items():
    if num == None:
        point = idx
        while point >= 0:
            #finds the nearest filled cell that is before it
            point -= 1
            val = dfTCB.iloc[point, 4]
            if val is not None:
                dfTCB.at[idx, "trial"] = val
                break
#first two must be manually done
dfTCB.at[0, "trial"] = 1
dfTCB.at[1, "trial"] = 1

In [31]:
dfTCB[5:15]

,block,exp_event,task,timestamp,trial
5,1,Image onset for trial 1 in block 1,Image onset,1.591225e+09,1
6,None,image onset,,1.591225e+09,1
7,None,testing_image_onset,,1.591225e+09,1
8,None,adjust,,1.591225e+09,1
9,1,Adjust Task begins for trial 1 in block 1,Adjust task,1.591225e+09,1
10,1,Adjust Task ends for trial 1 in block 1,,1.591225e+09,1
11,1,Ending trial 1 in block 1,End trial,1.591225e+09,1
12,1,Begin for trial 2 in block 1,Trial begins,1.591225e+09,2
13,1,Fixation dot begins for trial 2 in block 1,Fixation dot,1.591225e+09,2
14,None,fixation,,1.591225e+09,2


In [32]:
for idx, event in dfTCB.iterrows():
    #first 10 trials
    if float(dfTCB.iloc[idx, 4]) > 10:
        break
    #fixation
    if dfTCB.iloc[idx, 2] == "Fixation dot":
        task = "Fixation dot"
        trial = dfTCB.iloc[idx, 4]
        block = dfTCB.iloc[idx, 0]
        sent = dfTCB.iloc[idx, 3]
        if dfTCB.iloc[idx + 1, 1] == "fixation":
            received = dfTCB.iloc[idx + 1, 3]
        else:
            received = dfTCB.iloc[idx - 1, 3]
        row = [block, trial, task, sent, received]
        timesTCB.loc[len(timesTCB), :] = row
    #image onset
    elif dfTCB.iloc[idx, 2] == "Image onset":
        task = "Image onset"
        trial = dfTCB.iloc[idx, 4]
        block = dfTCB.iloc[idx, 0]
        sent = dfTCB.iloc[idx, 3]
        if dfTCB.iloc[idx + 1, 1] == "image onset":
            received = dfTCB.iloc[idx + 1, 3]
        else:
            received = dfTCB.iloc[idx - 1, 3]        
        row = [block, trial, task, sent, received]
        timesTCB.loc[len(timesTCB), :] = row
    elif dfTCB.iloc[idx, 2] == "Adjust task":
        task = "Adjust task"
        trial = dfTCB.iloc[idx, 4]
        block = dfTCB.iloc[idx, 0]
        sent = dfTCB.iloc[idx, 3]
        if dfTCB.iloc[idx + 1, 1] == "adjust":
            received = dfTCB.iloc[idx + 1, 3]
        else:
            received = dfTCB.iloc[idx - 1, 3]
        row = [block, trial, task, sent, received]
        timesTCB.loc[len(timesTCB), :] = row
    else:
        pass

In [33]:
timesTCB["delay"] = timesTCB["sent"] - timesTCB["received"]

In [34]:
avg = timesTCB["delay"].mean()

In [35]:
if avg < 0:
    timesTCB["fixedlag"] = timesTCB["received"] + avg #avg was negative
elif avg > 0:
    timesTCB["fixedlag"] = timesTCB["received"] - avg

In [36]:
timesTCB

,block,trial,task,sent,received,delay,fixedlag
0,1,1,Fixation dot,1.59123e+09,1.59123e+09,-0.0300457,1.59123e+09
1,1,1,Image onset,1.59123e+09,1.59123e+09,-0.139507,1.59123e+09
2,1,1,Adjust task,1.59123e+09,1.59123e+09,0.00205851,1.59123e+09
3,1,2,Fixation dot,1.59123e+09,1.59123e+09,-0.0401266,1.59123e+09
4,1,2,Image onset,1.59123e+09,1.59123e+09,-0.0865505,1.59123e+09
5,1,2,Adjust task,1.59123e+09,1.59123e+09,-0.00968146,1.59123e+09
6,1,3,Fixation dot,1.59123e+09,1.59123e+09,-0.0155714,1.59123e+09
7,1,3,Image onset,1.59123e+09,1.59123e+09,-0.0523214,1.59123e+09
8,1,3,Adjust task,1.59123e+09,1.59123e+09,-0.0153713,1.59123e+09
9,1,4,Fixation dot,1.59123e+09,1.59123e+09,-0.0435736,1.59123e+09


In [37]:
#for the binary SD marker
TCBbehavioral = pd.read_csv("/Users/selena_zhang/Desktop/000/TCBbehavioral.csv")

In [38]:
def SDlabel_trials1(behavioral_df):
    trial_unique = behavioral_df.trialNumber.unique()
    block_unique = behavioral_df.blockNumber.unique()
    behavioral_df["SD_CONDITION"] = ""
    row = 0
    for idx, col in behavioral_df.iterrows():
        if (behavioral_df.loc[row, "Stim_diff"] < 0) & (behavioral_df.loc[row, "responseError"] < 0):
            behavioral_df.set_value(row, "SD_CONDITION", "SD")
            row += 1
        elif (behavioral_df.loc[row, "Stim_diff"] > 0) & (behavioral_df.loc[row, "responseError"] > 0):
            behavioral_df.set_value(row, "SD_CONDITION", "SD")
            row += 1
        elif np.isnan(behavioral_df.loc[row, "Stim_diff"]) or np.isnan(behavioral_df.loc[row, "responseError"]):
            row += 1
        else:
            behavioral_df.set_value(row, "SD_CONDITION", "NO_SD")
            row += 1
    return behavioral_df

In [39]:
SDlabel_trials1(TCBbehavioral)

,stimulusID,morphID,RT,trialNumber,blockNumber,stimLocationDeg,shifted_stimulusID,shifted_stimLocationDeg,shifted_morphID,Error,responseError,Stim_diff,Stim_Loc_diff,bootstrap_values,Running_Mean,slopes,intercepts,DoVM,SD_CONDITION
0,81.0,86.0,1.214150,1.0,1.0,142.0,50.0,247.0,47.0,-3.0,-3.000000,31.0,-105.0,2.175724,-0.443572,0.164390,0.251218,-3.801264e-19,NO_SD
1,50.0,47.0,2.031325,2.0,1.0,247.0,77.0,65.0,81.0,4.0,3.997763,-27.0,-178.0,3.836778,-0.151244,0.256575,1.705796,-1.340351e-04,NO_SD
2,77.0,81.0,2.982389,3.0,1.0,65.0,145.0,220.0,125.0,-20.0,-20.000000,-68.0,-155.0,2.689834,-0.151244,0.303582,-0.711443,-2.708124e-04,SD
3,145.0,125.0,2.935164,4.0,1.0,220.0,119.0,165.0,103.0,-16.0,-16.000000,26.0,55.0,3.910172,-0.060910,0.356560,-0.611051,-4.131581e-04,NO_SD
4,119.0,103.0,2.170834,5.0,1.0,165.0,64.0,101.0,66.0,2.0,1.999554,55.0,64.0,0.716972,-0.083347,0.080926,0.218555,-5.640693e-04,SD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,46.0,51.0,4.196066,80.0,3.0,272.0,126.0,7.0,120.0,-6.0,-2.502850,66.0,-95.0,NaN,NaN,NaN,NaN,NaN,NO_SD
245,126.0,120.0,7.843110,81.0,3.0,7.0,62.0,274.0,42.0,-20.0,-20.000446,64.0,93.0,NaN,NaN,NaN,NaN,NaN,NO_SD
246,62.0,42.0,2.058722,82.0,3.0,274.0,5.0,108.0,13.0,8.0,8.003690,57.0,166.0,NaN,NaN,NaN,NaN,NaN,SD
247,5.0,13.0,3.374745,83.0,3.0,108.0,68.0,237.0,64.0,-4.0,-3.999999,-63.0,-129.0,NaN,NaN,NaN,NaN,NaN,SD


In [40]:
#cropping out subset that have blobs
timestampframe = pd.read_csv("/Users/selena_zhang/Desktop/000/preprocessed/pl_timestamps.csv", header=None)
messages = pd.read_csv("/Users/selena_zhang/Desktop/000/preprocessed/pl_msgs.csv")

In [41]:
timestampframe

,0
0,0.000000e+00
1,1.591225e+09
2,1.591225e+09
3,1.591225e+09
4,1.591225e+09
...,...
51626,1.591228e+09
51627,1.591228e+09
51628,1.591228e+09
51629,1.591228e+09


In [42]:
#timestamps of first trial blob
trial = 1
block = 1
begintask = "Image onset"
endtask = "Adjust task"
msg_idx = messages[(messages['task'] == begintask) & (messages['trial'] == trial) & (messages['block'] == block)].index.values
begin_ts = messages.loc[msg_idx[0], "timestamp"]
msg_end_idx = messages[(messages['task'] == endtask) & (messages['trial'] == trial) & (messages['block'] == block)].index.values
end_ts = messages.loc[msg_end_idx[0], 'timestamp']
#correct the lag
if avg > 0:
    begin_ts_lag = begin_ts - avg
    end_ts_lag = end_ts - avg
elif avg < 0:
    begin_ts_lag = begin_ts + avg
    end_ts_lag = end_ts + avg

In [43]:
def nearest_ts(time, ts_frame, task):
    #time = corrected time after delay, ts_frame=DF with timestamps, task = 'begin' or 'end'
    timediff = []
    for idx, ts in ts_frame.iloc[:, 0].items():
        diff = ts-time
        if diff < 0: 
            timediff.append(diff)
        elif diff > 0:
            if task=='begin':
                best_ts = timediff[-1]
                best_frame = idx + 1
            elif task=='end':
                best_ts = ts
                best_frame = idx + 2
            break
        else: 
            best_frame = idx + 1
            break
    return best_frame


In [44]:
#best frame number
begin_frame = nearest_ts(begin_ts_lag, timestampframe, 'begin')
end_frame = nearest_ts(end_ts_lag, timestampframe, 'end')

In [45]:
#subset out frames from one trial

if platform.system() == 'Darwin':
    EM_path = '/Users/selena_zhang/Desktop/000/'    
elif platform.system() == 'Linux':
    EM_path = '/media/whitney/New Volume/Teresa/SD_grant_EM/TCB_GOOD/000/'

subset_path = EM_path + 'trial1frames'
frames_path = EM_path + 'frames'

try:
    if not os.path.exists(subset_path):
        os.mkdir(subset_path)
        print("Successfully created the directory %s " % subset_path)
    else:
        print("Directory already exists %s " % subset_path)
except OSError:
    print("Creation of the directory %s failed" % subset_path)

Directory already exists /Users/selena_zhang/Desktop/000/trial1frames 


In [46]:
#copy files into trial1frames from frames (full)
for i in range(begin_frame, end_frame + 1):
    if os.path.isfile(subset_path + str(i) + ".png"):
        pass
    else:
        copyfile(frames_path + "/frame" + str(i) + ".png", subset_path + "/frame" + str(i) + ".png")

In [47]:
correct_tags = [0, 1, 2, 3, 5, 6, 7, 8, 9, 11]

In [64]:
#make new df
blob_coords = pd.DataFrame(columns=["trial", "frame", "timestamp", "topleft", "topright", "bottomleft", "bottomright", "center"])
#run find_blob on frames in subset
def blob_finder(bframe, eframe, trial, block):
    degree = TCBbehavioral[(TCBbehavioral["trialNumber"] == trial) & (TCBbehavioral['blockNumber'] == block)].index.values[0]
    for i in range(bframe, eframe + 1): 
        blob = find_blob(subset_path + "/frame" + str(i) + ".png", correct_tags, degree)
        tright= (blob[2][0], blob[1][1])
        bleft = (blob[1][0], blob[2][1])
        values = [trial, i, timestampframe.iloc[i+1, 0], blob[1], tright, bleft, blob[2], blob[0]]
        blob_coords.loc[len(blob_coords), :] = values
blob_finder(begin_frame, end_frame, 1, 1)

Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |█████████████████████████████████████████

In [65]:
blob_coords

,trial,frame,timestamp,topleft,topright,bottomleft,bottomright,center
0,1,4016,1.59123e+09,"(616, 235)","(696, 235)","(616, 315)","(696, 315)","(656.0, 275.0)"
1,1,4017,1.59123e+09,"(617, 233)","(697, 233)","(617, 313)","(697, 313)","(657.0, 273.0)"
2,1,4018,1.59123e+09,"(617, 233)","(697, 233)","(617, 313)","(697, 313)","(657.0, 273.0)"
3,1,4019,1.59123e+09,"(617, 233)","(697, 233)","(617, 313)","(697, 313)","(657.0, 273.0)"
4,1,4020,1.59123e+09,"(617, 233)","(697, 233)","(617, 313)","(697, 313)","(657.0, 273.0)"
...,...,...,...,...,...,...,...,...
62,1,4078,1.59123e+09,"(614, 240)","(694, 240)","(614, 320)","(694, 320)","(654.0, 280.0)"
63,1,4079,1.59123e+09,"(614, 241)","(694, 241)","(614, 321)","(694, 321)","(654.0, 281.0)"
64,1,4080,1.59123e+09,"(615, 241)","(695, 241)","(615, 321)","(695, 321)","(655.0, 281.0)"
65,1,4081,1.59123e+09,"(615, 241)","(695, 241)","(615, 321)","(695, 321)","(655.0, 281.0)"
